In [2]:
import requests
import pandas as pd
from datetime import datetime
import pandas as pd

In [21]:
earliest_complaint = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintdate FROM complaints order by complaintdate limit 1
"""
).json()["rows"][0]["complaintdate"]

latest_complaint = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintdate FROM complaints order by complaintdate desc limit 1
"""
).json()["rows"][0]["complaintdate"]

n_complaints = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM complaints
"""
).json()["rows"][0]["count"]
print(
    f"There have been {n_complaints} complaints between {earliest_complaint} and {latest_complaint}"
)

There have been 695293 complaints between 2000-03-03T16:15:00Z and 2020-12-22T00:00:00Z


In [22]:
earliest_maint_complaint = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintdate FROM complaints where complaintcodename like '%MAINT%' order by complaintdate limit 1
"""
).json()["rows"][0]["complaintdate"]

latest_maint_complaint = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintdate FROM complaints where complaintcodename like '%MAINT%'order by complaintdate desc limit 1
"""
).json()["rows"][0]["complaintdate"]

n_maint_complaints = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT count(*) FROM complaints where complaintcodename like '%MAINT%'
"""
).json()["rows"][0]["count"]
print(
    f"There have been {n_maint_complaints} complaints ({n_maint_complaints/n_complaints*100:.2f}% of total) between {earliest_maint_complaint} and {latest_maint_complaint}"
)

There have been 218455 complaints (31.42% of total) between 2007-01-02T09:14:00Z and 2020-12-18T00:00:00Z


In [23]:
complaints = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT * FROM complaints limit 5
"""
).json()["rows"]
complaints[0]

{'cartodb_id': 1,
 'the_geom': '0101000020E6100000F3819B81EDCA52C0335388EDED044440',
 'the_geom_webmercator': '0101000020110F00007E415D4CD8EB5FC1763ADB0A5D955241',
 'objectid': 20,
 'addressobjectid': '31611',
 'address': '5613 BAYNTON ST',
 'unit_type': None,
 'unit_num': None,
 'zip': '19144-2210',
 'censustract': '247',
 'parcel_id_num': '59236',
 'opa_account_num': '122153100',
 'opa_owner': 'LANE ROY',
 'complaintnumber': '233230',
 'casenumber': None,
 'complaintcode': 'BD',
 'complaintcodename': 'BUILDING DANGEROUS',
 'unitresponsible': None,
 'ticket_num_311': None,
 'complaintdate': '2007-01-02T10:37:00Z',
 'complaintstatus': None,
 'casestatus': None,
 'initialinvestigation_date': '2007-01-04T07:29:00Z',
 'complaintresolution_date': None,
 'systemofrecord': 'HANSEN',
 'geocode_x': 2690592.05329031,
 'geocode_y': 267431.98469711}

In [11]:
complaint_status_results = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintstatus, count(*) FROM complaints group by complaintstatus
"""
).json()["rows"]
complaint_status_counts_df = pd.DataFrame(complaint_status_results).sort_values(
    "count", ascending=False
)
complaint_status_counts_df

,complaintstatus,count
4,None,646400
1,CLOSED,30448
2,IN REVIEW,18297
0,CANCELLED,104
3,NEW,44


In [25]:
complaint_codename_results = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintcodename as complaintcodename_like_MAINT, count(*) FROM complaints where complaintcodename like '%MAINT%'  group by complaintcodename
"""
).json()["rows"]
complaint_codename_counts_df = pd.DataFrame(complaint_codename_results).sort_values(
    "count", ascending=False
)
complaint_codename_counts_df

,complaintcodename_like_maint,count
3,MAINTENANCE RESIDENTIAL,169545
2,MAINTENANCE COMMERCIAL,17046
1,DRAINAGE MAINTENANCE RESIDENTIAL TO HCEU,12568
4,PROPERTY MAINTENANCE COMPLAINT EXTERIOR,9063
6,PROPERTY MAINTENANCE HIGH WEEDS,5491
5,PROPERTY MAINTENANCE COMPLAINT INTERIOR,4183
0,DRAINAGE MAINTENANCE COMMERCIAL TO CI,546
8,SPECIAL MAINTENANCE RESIDENTIAL,10
7,SPECIAL MAINTENANCE COMMERCIAL,3


In [36]:
nonmatching_complaints = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT address, complaintnumber, complaintstatus, complaintdate, opa_owner, opa_account_num
    FROM complaints c where c.opa_account_num not in (select parcel_number from opa_properties_public)
"""
).json()["rows"]
nonmatching_complaints_df = pd.DataFrame(nonmatching_complaints)
print(
    f"It appears that there exists {len(nonmatching_complaints)} error(s) in the data, but otherwise complaints can be traced to properties."
)
nonmatching_complaints_df

It appears that there exists 15 error(s) in the data, but otherwise complaints can be traced to properties.


,address,complaintnumber,complaintstatus,complaintdate,opa_owner,opa_account_num
0,1325 N MARSTON ST,CM-2020-011379,CLOSED,2020-04-30T00:00:00Z,1325 N MARSTON ST LLC,292072005
1,523-25 N BROAD ST,CM-2020-053959,CLOSED,2020-08-21T00:00:00Z,523 NORTH BROAD OWNER LP,881041455
2,456 N 5TH ST,CM-2020-056375,CLOSED,2020-08-25T00:00:00Z,None,884028600
3,456 N 5TH ST,CM-2020-056376,CLOSED,2020-08-25T00:00:00Z,None,884028600
4,456 N 5TH ST,CM-2020-056377,CLOSED,2020-08-25T00:00:00Z,None,884028600
5,456 N 5TH ST,CM-2020-055954,CLOSED,2020-08-27T00:00:00Z,None,884028600
6,456 N 5TH ST,CM-2020-055956,CLOSED,2020-08-26T00:00:00Z,None,884028600
7,456 N 5TH ST,CM-2020-055957,CLOSED,2020-08-25T00:00:00Z,None,884028600
8,456 N 5TH ST,CM-2020-055962,CLOSED,2020-08-25T00:00:00Z,None,884028600
9,456 N 5TH ST,CM-2020-055964,CLOSED,2020-08-25T00:00:00Z,None,884028600


In [29]:
matching_complaints_result = requests.get(
    """
    https://phl.carto.com/api/v2/sql?q=
    SELECT c.complaintnumber, c.complaintcodename, c.complaintdate, c.complaintstatus, c.casestatus, c.initialinvestigation_date, c.complaintresolution_date, 
    c.systemofrecord,
    opp.parcel_number, opp.mailing_address_1, opp.mailing_address_2, opp.mailing_street, opp.mailing_city_state, opp.owner_1, opp.owner_2, opp.location
    FROM complaints c, opa_properties_public opp where c.opa_account_num= opp.parcel_number limit 1000
    
"""
).json()["rows"]

In [31]:
pd.DataFrame(matching_complaints_result).head()

,complaintnumber,complaintcodename,complaintdate,complaintstatus,casestatus,initialinvestigation_date,complaintresolution_date,systemofrecord,parcel_number,mailing_address_1,mailing_address_2,mailing_street,mailing_city_state,owner_1,owner_2,location
0,233230,BUILDING DANGEROUS,2007-01-02T10:37:00Z,None,None,2007-01-04T07:29:00Z,None,HANSEN,122153100,None,None,None,None,LANE ROY,None,5613 BAYNTON ST
1,233333,VACANT HOUSE,2007-01-03T09:17:00Z,None,None,2007-09-04T04:35:00Z,2007-09-04T04:35:00Z,HANSEN,886505500,None,None,5429 CHESTNUT ST STE M114,PHILADELPHIA PA,NEIGHBORHOOD RESTORATIONS,LIMITED PARTNERSHIP IV,6001 CLIFFORD TER
2,233231,MAINTENANCE RESIDENTIAL,2007-01-02T10:38:00Z,None,None,2008-02-01T15:54:00Z,2008-02-01T15:55:00Z,HANSEN,012477500,None,None,2314 VISTA ST,PHILADELPHIA PA,YANG GINA,None,1434 S 12TH ST
3,233334,BUILDING CONSTRUCTION,2007-01-03T09:18:00Z,None,None,2007-01-03T12:04:00Z,2007-01-03T12:04:00Z,HANSEN,631098800,None,None,None,None,YANG RAE SUK,YANG SOON JA,7319 SHELBOURNE ST
4,233338,INFESTATION RESIDENTIAL,2007-01-03T09:25:00Z,None,None,2007-08-29T07:49:00Z,2007-08-29T07:49:00Z,HANSEN,886760980,PHILA HOUSING AUTHORITY,EXECUTIVE OFFICES 6TH FL,12 S 23RD ST,PHILADELPHIA PA,PHILA HOUSING AUTHORITY,None,5600 LINDBERGH BLVD


### Other notes about this data

In [15]:
complaint_codename_results = requests.get(
    """
https://phl.carto.com/api/v2/sql?q=SELECT complaintcodename, count(*) FROM complaints group by complaintcodename
"""
).json()["rows"]
complaint_codename_counts_df = pd.DataFrame(complaint_codename_results).sort_values(
    "count", ascending=False
)
complaint_codename_counts_df

,complaintcodename,count
57,MAINTENANCE RESIDENTIAL,169545
16,COMMUNITY LIFE IMPROVEMENT PROGRAM,84181
105,VACANT HOUSE,63979
8,BUILDING CONSTRUCTION,59622
110,VACANT LOTS (CLIP),52717
...,...,...
42,GRAFFITI,1
50,LARGE VACANT,1
3,BLOCKED DRAIN COMMERCIAL,1
88,SPECIAL CIRCUMSTANCE,1


The community life improvement program [CLIP](https://www.phila.gov/departments/community-life-improvement-program/)  (second most complaints) focuses on removing graffiti and cleaning up vacant lots.